In [2]:
import pandas as pd
import numpy as np 
import glob
from biopandas.pdb import PandasPdb
import warnings
warnings.filterwarnings("ignore")
from get_distances import *
import sys
from multiprocessing import Pool
import time
import math
import datetime

In [56]:
psp_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/test_psp.csv").sample(n=5)
output_location = "/people/imal967/git_repos/pheno_analysis/merged_pockets.csv"

In [57]:
def run_parallel_pockets(number_of_threads, psp_data):
    #adding columns to psp data

    psp_data['closest_pocket'] = ""
    psp_data['inside_pocket'] = 0
    psp_data['distance_from_pocket'] = np.nan

    # get all of the unique uniprots
    unique_uniprot_psp = [psp_data[psp_data["uniprot_id"]==uniprot_id].copy() for uniprot_id in psp_data["uniprot_id"].unique()]


    start_time = time.perf_counter()
    with Pool(10) as pool:
        output = pool.map(find_pockets_per_uniprot, unique_uniprot_psp)
    finish_time = time.perf_counter()
    
    # save the csv and output the start and end times
    print("Program finished in {} seconds - using multiprocessing with {} cores".format(str(datetime.timedelta(seconds=finish_time-start_time)), number_of_threads))
    concatenated_output = pd.concat(output)
    return(concatenated_output)



In [58]:
'''
this function does pockets calcuations for each uniprot tthat it is given
'''

# for each unique uniprotID...
# for uniprot in unique_uniprots:
def find_pockets_per_uniprot(psp_only_uniprot):
    #print("start")
    # isolate to psp and pockets in each uniprot
    pockets_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/pockets_data.csv")
    uniprot = psp_only_uniprot["uniprot_id"].to_list()[0]
    pocket_only_uniprot = pockets_data[pockets_data['uniprot_id'] == uniprot]


    # parse your structure here
    pdb_name = glob.glob("/rcfs/projects/proteometer/alphafold_swissprot_pdb/*" + uniprot + "-F1-*")
    print("name of pdb is:", pdb_name)
    if pdb_name:  
        ppdb = PandasPdb()  
        ppdb.read_pdb(pdb_name[0])


    # for each psp
        for phosphosite_row_index in psp_only_uniprot.index:
            #print(psp_only_uniprot)
            #print(phosphosite_row_index)
            residue_num = int(psp_only_uniprot.loc[phosphosite_row_index,'res_number']) # finding the residue number of the psp
            min_dist = np.inf # make min dist extremely high at first
            #print(residue_num)
            # use the residue # to get the coordinates in space from pdb file
            
            
            for pocket_index in pocket_only_uniprot.index : # get all the residues in all of the pockets 
                if pd.notna(pocket_only_uniprot.loc[pocket_index,'pocket_resid']):
                    pocket_residues = pocket_only_uniprot.loc[pocket_index,'pocket_resid']

                    # format the residues
                    pocket_residues = [int(e) for e in pocket_residues[1:-1].split(",")]
                    #print(pocket_residues)
                    if residue_num in pocket_residues:
                        psp_data.loc[phosphosite_row_index,'inside_pocket'] = 1 # if residue is in the pocket, put 1 in the inside pocket column
                        psp_data.loc[phosphosite_row_index,'closest_pocket'] = pocket_only_uniprot.loc[pocket_index,'full_id'] # put unique pocketID in closest pocket
                        psp_data.loc[phosphosite_row_index,'distance_from_pocket'] = 0 
                        break # break because you don't want to contiue looking for pockets (and therefore overwrite the inside pocket and closest pocket)
                    else: # if the phosphosite isn't in any pockets
                        #print("phosphosite isn't in any pockets")
                        input_struct = ppdb.df['ATOM']
                        #print(input_struct)
                        print(residue_num)
                        print(pocket_residues)
                        new_dist = find_mean_distances(input_struct, residue_num, pocket_residues)
                        # print(new_dist)
                        if new_dist:
                            if min_dist > new_dist: # if this is the smallest distance so far, replace min_dist with new_dist
                                psp_data.loc[phosphosite_row_index,'closest_pocket'] = pocket_only_uniprot.loc[pocket_index,'full_id'] # put unique pocketID in closest pocket
                                psp_data.loc[phosphosite_row_index,'distance_from_pocket'] = new_dist # replace distance_from_pocket with min_dist
                                min_dist = new_dist 
                                print("added smallest distance:", min_dist)


    #print("end")

    return(psp_data)

In [59]:
please = run_parallel_pockets(4,psp_data)
please

name of pdb is: ['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-Q96GE5-F1-model_v4.pdb']
599
[3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 18, 19, 29, 30, 33, 34, 37, 40, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56]
name of pdb is: ['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-P53671-F1-model_v4.pdb']
added smallest distance: 78.67230231372041
599
[109, 110, 111, 112, 113, 114, 115, 148, 149, 150, 151, 152, 154, 155, 156, 158, 159, 160, 162, 179, 180, 183]
151
[38, 40, 43, 45, 46, 47, 48, 49, 50, 51, 52, 60, 61, 62, 65, 78, 80, 81, 82, 335, 337, 338, 339, 340, 341, 342, 343, 344, 345, 347, 358, 359, 360, 368, 372, 376, 380, 389, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 415, 451, 453, 455, 456, 457, 458, 468, 469, 470, 471, 472, 502, 503, 504, 505, 506, 507, 508, 509, 510, 513, 516, 517, 518, 519, 520, 521, 522, 523, 544]
name of pdb is: ['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-P07942-F1-model_v4.pdb']
599
[605, 607, 608, 609, 611, 612, 613, 614, 615, 

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_pocket,inside_pocket,distance_from_pocket
25,122781,LIMK2,LIMK2,P53671,22q12.2,T151-p,4949691,human,72.23,NaN,...,P53671,THR,151.0,NaN,undefined,151.0,P53671,P53671-F1_pocket13,1,0.000000
11,260037,XRCC1,XRCC1,P18887,19q13.31,S418-p,3181471,human,69.50,NaN,...,P18887,SER,418.0,NaN,undefined,418.0,P18887,,0,NaN
45,120300,LAMB1,LAMB1,P07942,7q31.1,Y220-p,20621765,human,198.04,Laminin_N,...,P07942,TYR,220.0,13.47212,protonated,220.0,P07942,,0,NaN
42,270700,ZNF799,ZNF799,Q96GE5,19p13.2,S599-p,23077152,human,74.29,zf-C2H2_6,...,Q96GE5,SER,599.0,NaN,undefined,599.0,Q96GE5,,0,NaN
26,115679,KIAA1549,KIAA1549,Q9HCM3,7q34,S1892-p,14160343,human,210.76,NaN,...,Q9HCM3,SER,1892.0,NaN,undefined,1892.0,Q9HCM3,,0,NaN
25,122781,LIMK2,LIMK2,P53671,22q12.2,T151-p,4949691,human,72.23,NaN,...,P53671,THR,151.0,NaN,undefined,151.0,P53671,,0,NaN
11,260037,XRCC1,XRCC1,P18887,19q13.31,S418-p,3181471,human,69.50,NaN,...,P18887,SER,418.0,NaN,undefined,418.0,P18887,P18887-F1_pocket8,0,29.038891
45,120300,LAMB1,LAMB1,P07942,7q31.1,Y220-p,20621765,human,198.04,Laminin_N,...,P07942,TYR,220.0,13.47212,protonated,220.0,P07942,,0,NaN
42,270700,ZNF799,ZNF799,Q96GE5,19p13.2,S599-p,23077152,human,74.29,zf-C2H2_6,...,Q96GE5,SER,599.0,NaN,undefined,599.0,Q96GE5,,0,NaN
26,115679,KIAA1549,KIAA1549,Q9HCM3,7q34,S1892-p,14160343,human,210.76,NaN,...,Q9HCM3,SER,1892.0,NaN,undefined,1892.0,Q9HCM3,,0,NaN


## Testing Interfaces Parallel

In [21]:
psp_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/test_psp.csv").sample(n = 5)
output_location = "/people/imal967/git_repos/pheno_analysis/merged_interfaces.csv"
psp_data

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position,uniprot_id
5,313274,CCNYL1,CCNYL1,Q8N7R7,2q33.3,K176-ub,964504078,human,40.71,Cyclin_N,...,0,176,Q8N7R7_176,Q8N7R7,LYS,176.0,10.650554,protonated,176.0,Q8N7R7
22,287242,NDN,NDN,Q99608,15q11.2,K126-ac,6218405,human,36.09,NaN,...,0,126,Q99608_126,Q99608,LYS,126.0,10.870360,protonated,126.0,Q99608
37,300955,AHNAK,AHNAK,Q09666,11q12.3,K891-ub,15384061,human,629.10,NaN,...,0,891,Q09666_891,Q09666,ASP,891.0,3.798373,deprotonated,1491.0,Q09666
41,184191,PTMA,PTMA,P06454,2q37.1,S9-p,11537664,human,12.20,Prothymosin,...,0,9,P06454_9,P06454,SER,9.0,NaN,undefined,9.0,P06454
1,235777,TTN,Titin,Q8WZ42,2q31.2,S314-p,6147144,human,3816.03,NaN,...,0,314,Q8WZ42_314,Q8WZ42-F5,GLU,314.0,3.907562,deprotonated,10914.0,Q8WZ42


In [25]:
def run_parallel_interfaces(number_of_threads):
    #adding columns to psp data

    psp_data['closest_interface'] = ""
    psp_data['inside_interface'] = 0
    psp_data['distance_from_interface'] = np.nan

    # get all of the unique uniprots
    # unique_uniprots = psp_data['uniprot_id'].unique()
    unique_uniprot_psp = [psp_data[psp_data["uniprot_id"]==uniprot_id].copy() for uniprot_id in psp_data["uniprot_id"].unique()]

    start_time = time.perf_counter()
    with Pool(number_of_threads) as pool:
        output = pool.map(find_interfaces_per_uniprot, unique_uniprot_psp)
    finish_time = time.perf_counter()
    
    # save the csv and output the start and end times
    print("Program finished in {} seconds - using multiprocessing with {} cores".format(str(datetime.timedelta(seconds=finish_time-start_time)), number_of_threads))
    concatenated_output = pd.concat(output)
    return(concatenated_output)


In [19]:
'''
this function does interfaces calcuations for each uniprot that it is given
'''

# for each unique uniprotID...
# for uniprot in unique_uniprots:
def find_interfaces_per_uniprot(psp_only_uniprot):

    # isolate to psp and interfaces in each uniprot
    interfaces_data = pd.read_csv("/rcfs/projects/proteometer/ProtVar/predictions/interfaces/2024.05.28_interface_summary_5A.tsv", delimiter='\t', header=0)
    uniprot = psp_only_uniprot["uniprot_id"].to_list()[0]
    interface_only_uniprot = interfaces_data.loc[(interfaces_data['uniprot_id1'] == uniprot) | (interfaces_data['uniprot_id2'] == uniprot)] # isolate to uniprot in either 1 or 2


    # parse your structure here
    pdb_name = glob.glob("/rcfs/projects/proteometer/alphafold_swissprot_pdb/*" + uniprot + "-F1-*") # change this to have F1 using pdb file name
    print(pdb_name)
    #print("name of pdb is:", pdb_name)
    if len(pdb_name) != 0:  
        ppdb = PandasPdb()  
        ppdb.read_pdb(pdb_name[0])


    # for each psp
        for phosphosite_row_index in psp_only_uniprot.index:
            residue_num = int(psp_only_uniprot.loc[phosphosite_row_index,'res_number']) # finding the residue number of the psp
            min_dist = np.inf # make min dist extremely high at first
            #print(residue_num)
            # use the residue # to get the coordinates in space from pdb file
            
            
            for interface_index in interface_only_uniprot.index : # get all the residues in all of the interfaces 
                if pd.notna(interface_only_uniprot.loc[interface_index,'ifresid1']) & pd.notna(interface_only_uniprot.loc[interface_index,'ifresid2']):
                    if interfaces_data.loc[interface_index,'uniprot_id1'] == uniprot:
                        interface_residues = interface_only_uniprot.loc[interface_index,'ifresid1']
                    elif interfaces_data.loc[interface_index,'uniprot_id2'] == uniprot:
                        interface_residues = interface_only_uniprot.loc[interface_index,'ifresid2']
                    
                    # check if it's inside of a interface
                    #print(interfaces_data.loc[interface_index,'interaction_id']) 
                    interface_residues = [int(e[1:]) for e in interface_residues.split(",")] # remove the first letter from each bc it includes residue type ormat the interface_residues because it's a string
                    print(interface_residues)
                    if residue_num in interface_residues:
                        #print("found inside interface!")
                        psp_only_uniprot.loc[phosphosite_row_index,'inside_interface'] = 1 # if residue is in the interface, put 1 in the inside interface column
                        interface_to_add = (interface_only_uniprot.loc[interface_index,'interaction_id'].split('_'))
                        interface_to_add.remove(uniprot)
                        psp_only_uniprot.loc[phosphosite_row_index,'closest_interface'] = interface_to_add[0] # put unique interfaceID in closest interface
                        psp_only_uniprot.loc[phosphosite_row_index,'distance_from_interface'] = 0.0 
                        break # break because you don't want to contiue looking for interfaces (and therefore overwrite the inside interface and closest interface)
                    else: # if the phosphosite isn't in any interfaces
                        #print("phosphosite isn't in any interfaces")
                        input_struct = ppdb.df['ATOM']
                        #print(input_struct)
                        
                        #print(residue_num, interface_residues)
                        new_dist = find_mean_distances(input_struct, residue_num, interface_residues)
                        print("the new dist is:" , new_dist)
                        if new_dist:
                            if min_dist > new_dist: # if this is the smallest distance so far, replace min_dist with new_dist
                                interface_to_add = (interface_only_uniprot.loc[interface_index,'interaction_id'].split('_'))
                                interface_to_add.remove(uniprot)
                                psp_only_uniprot.loc[phosphosite_row_index,'closest_interface'] = interface_to_add[0]
                                psp_only_uniprot.loc[phosphosite_row_index,'distance_from_interface'] = new_dist # replace distance_from_interface with min_dist
                                min_dist = new_dist
                                print("replaced old dist with", min_dist)
    return(psp_only_uniprot)



In [28]:
please = run_parallel_interfaces(1)
please

['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-Q8N7R7-F1-model_v4.pdb']
[63, 64, 65, 66, 67, 68, 69, 70, 80, 162, 222, 223, 224, 225, 242, 245, 246, 247, 248, 249, 251, 253, 254, 255, 266, 267, 270, 273, 274, 277, 278, 279, 281, 282, 284, 285]
the new dist is: 32.13138060232234
replaced old dist with 32.13138060232234
[62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 160, 222, 223, 224, 225, 242, 245, 246, 247, 252, 253, 254, 255, 266, 267, 270, 271, 273, 274, 277, 278, 279, 281, 282, 284, 285]
the new dist is: 32.22969685197742
[139, 196, 203, 205, 242, 245, 246, 252, 253, 273, 277, 284]
the new dist is: 28.870750832014878
replaced old dist with 28.870750832014878
[200, 203]
the new dist is: 27.338792745234166
replaced old dist with 27.338792745234166
[133, 134, 207, 245, 247, 248, 250, 251, 253, 286, 288, 291]
the new dist is: 35.50421429140231
[242, 245, 270, 273, 274, 277, 284]
the new dist is: 27.303841507645643
replaced old dist with 27.303841507645643
[204, 205, 266, 270, 274, 2

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_interface,inside_interface,distance_from_interface
5,313274,CCNYL1,CCNYL1,Q8N7R7,2q33.3,K176-ub,964504078,human,40.71,Cyclin_N,...,Q8N7R7,LYS,176.0,10.650554,protonated,176.0,Q8N7R7,Q86VU5,0,23.548959
22,287242,NDN,NDN,Q99608,15q11.2,K126-ac,6218405,human,36.09,NaN,...,Q99608,LYS,126.0,10.870360,protonated,126.0,Q99608,P04637,1,0.000000
37,300955,AHNAK,AHNAK,Q09666,11q12.3,K891-ub,15384061,human,629.10,NaN,...,Q09666,ASP,891.0,3.798373,deprotonated,1491.0,Q09666,,0,NaN
41,184191,PTMA,PTMA,P06454,2q37.1,S9-p,11537664,human,12.20,Prothymosin,...,P06454,SER,9.0,NaN,undefined,9.0,P06454,P11142,0,5.658883
1,235777,TTN,Titin,Q8WZ42,2q31.2,S314-p,6147144,human,3816.03,NaN,...,Q8WZ42-F5,GLU,314.0,3.907562,deprotonated,10914.0,Q8WZ42,,0,NaN


In [14]:
uniprot = "Q99595"
interfaces_data = pd.read_csv("/rcfs/projects/proteometer/ProtVar/predictions/interfaces/2024.05.28_interface_summary_5A.tsv", delimiter='\t', header=0)
interfaces_data.loc[(interfaces_data['uniprot_id1'] == uniprot) | (interfaces_data['uniprot_id2'] == uniprot)]

,interaction_id,pdockq,uniprot_id1,uniprot_id2,chain1,chain2,ifresid1,ifresid2,sources,n_references,pdb
7059,Q99595_Q9BVV7,0.65,Q99595,Q9BVV7,A,B,"P8,R12,D16,D77,V81,Q82,R84,G85,K86,E87,D88,P89...","F125,T127,I128,F129,E131,L132,F133,S137,P138,S...","corum,otar,string",0,Q99595/Q99595_Q9BVV7.pdb
9291,O94826_Q99595,0.62,O94826,Q99595,A,B,NaN,NaN,"otar,string",0,O94826/O94826_Q99595.pdb
12809,O14925_Q99595,0.59,O14925,Q99595,A,B,"E74,F77,F78,I80,G81,G82,C84,M85,A128,N129,G132...","W11,I13,V14,D15,C17,G18,G19,F21,T22,M23,T25,I2...","corum,otar,string",0,O14925/O14925_Q99595.pdb
17270,Q99595_Q9Y512,0.55,Q99595,Q9Y512,A,B,NaN,NaN,"otar,string",0,Q99595/Q99595_Q9Y512.pdb
26370,Q3ZCQ8_Q99595,0.49,Q3ZCQ8,Q99595,A,B,"T155,P161,W163,L165,G168,W169,F171,E196,T197,G...","E3,Y4,A5,R6,E7,P8,W11,T134,R135,A137,S138,F141...","corum,otar,string",0,Q3ZCQ8/Q3ZCQ8_Q99595.pdb
30256,O75390_Q99595,0.47,O75390,Q99595,A,B,NaN,NaN,"otar,string",0,O75390/O75390_Q99595.pdb
31581,P60602_Q99595,0.46,P60602,Q99595,A,B,"M25,V29,A32,A33,L36,F37,G65,T66,F67,T69,F70,M7...","G19,T22,M23,G24,I26,G27,G28,I30,F31,L62,F66,M1...","BioGRID,corum,intact,otar,string",1,P60602/P60602_Q99595.pdb
59222,Q15070_Q99595,0.35,Q15070,Q99595,A,B,"P139,W141,W251,W252,Q254,P261,I262","M23,G27,L62,S65,F66,W69,V113,A116,A117,L123",otar,0,Q15070/Q15070_Q99595.pdb
81274,P43361_Q99595,0.29,P43361,Q99595,A,B,"D113,E114,A117,E118,V120,R121,K156,E159,C160,V...","S115,A116,M118,G119,I121,L122,L123,A124,L125,I...",BioGRID,1,P43361/P43361_Q99595.pdb
83596,Q71U36_Q99595,0.28,Q71U36,Q99595,A,B,E423,A111,"BioGRID,intact",1,Q71U36/Q71U36_Q99595.pdb


In [17]:
uniprot = "Q8IYT2"
pdb_name = glob.glob("/rcfs/projects/proteometer/alphafold_swissprot_pdb/*" + uniprot + "*")
ppdb = PandasPdb()  
ppdb.read_pdb(pdb_name[0])
input_struct = ppdb.df['ATOM']
residue_num = 525.0
interface_residues = ['0']
atom_1 = input_struct.query('residue_number == 770 and atom_name == "CA"')
atom_2 = input_struct.query('residue_number == 77 and atom_name == "CA"')
if atom_1['x_coord'].values:
    x_p, y_p, z_p = atom_1['x_coord'].values, atom_1['y_coord'].values, atom_1['z_coord'].values
    print(x_p, y_p, z_p)
    x_q, y_q, z_q  = atom_2['x_coord'].values, atom_2['y_coord'].values, atom_2['z_coord'].values
else:
    print("False")
#find_mean_distances(input_struct, residue_num, interface_residues)
distance = np.sqrt((x_p - x_q)**2 + (y_p - y_q)**2 + (z_p - z_q)**2)[0]
distance

[1.793] [42.984] [-33.188]


np.float64(43.71312061612623)

In [67]:
interfaces_data

,interaction_id,pdockq,uniprot_id1,uniprot_id2,chain1,chain2,ifresid1,ifresid2,sources,n_references,pdb
0,O75106_Q16853,0.74,O75106,Q16853,A,B,"R169,A203,A204,V205,H206,L212,R213,W220,N226,I...","P39,V209,L218,Q219,W226,N232,I233,S234,G235,A2...","BioGRID,humap,intact,string",2,O75106/O75106_Q16853.pdb
1,Q15118_Q15118,0.73,Q15118,Q15118,A,B,"S53,P54,P56,Y179,D182,R183,M186,L255,A257,H304...","S53,P54,P56,Y179,D182,R183,M186,E253,L255,A257...","BioGRID,intact",2,Q15118/Q15118_Q15118.pdb
2,P11142_Q92598,0.73,P11142,Q92598,A,B,"K25,E27,I28,A30,N31,D32,Q33,G34,R36,E48,L50,D5...","R19,A27,N28,E29,F30,S31,R33,N54,T58,Y184,R261,...","BioGRID,corum,humap,intact,otar,string,xlinkdb",9,P11142/P11142_Q92598.pdb
3,Q13326_Q16585,0.73,Q13326,Q16585,A,B,"V40,L41,L43,L44,L47,V48,N50,L51,T54,I55,L58,F6...","V68,I69,L71,L72,L75,A76,I78,N79,I82,I86,M100,F...","corum,otar,string",0,Q13326/Q13326_Q16585.pdb
4,Q13326_Q92629,0.73,Q13326,Q92629,A,B,"K33,L36,Y37,V40,L41,L43,L44,L47,V48,N50,L51,T5...","R30,K31,C33,L34,F37,V38,L40,L41,L44,I45,V47,N4...","corum,string",0,Q13326/Q13326_Q92629.pdb
...,...,...,...,...,...,...,...,...,...,...,...
486094,P23193_Q92889,0.00,NaN,NaN,NaN,NaN,NaN,NaN,otar,0,NaN
486095,P23193_Q92541,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"BioGRID,intact,otar,string",1,NaN
486096,P23193_Q8WX92,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"otar,string",0,NaN
486097,P23193_Q8WVC0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,"BioGRID,intact,otar,string",2,NaN


In [75]:
type(psp_data['res_number'].to_list()[0])

float

In [23]:
psp_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/phosphosite_for_pockets.csv")
null_rows = psp_data[psp_data['res_number'].isnull()]
null_rows

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,Ambiguous_Site,RES_NUM,PKA_ID,uniprotID,AA,res_number,pK,state,position,uniprot_id
303,303,EIF4ENIF1,4E-T iso2,Q9NRA8-2,22q12.2,S436-p,22947401,human,88.23,EIF4E-T,...,0,436,Q9NRA8-2_436,NaN,NaN,NaN,NaN,NaN,NaN,Q9NRA8
352,352,HTR3D,5-HT(3D) iso4,Q70Z44-4,3q27.1,T120-p,25258606,human,45.15,NaN,...,0,120,Q70Z44-4_120,NaN,NaN,NaN,NaN,NaN,NaN,Q70Z44
354,354,HTR3E,5-HT(3E) iso3,A5X5Y0-3,3q27.1,T10-p,22953107,human,52.81,NaN,...,0,10,A5X5Y0-3_10,NaN,NaN,NaN,NaN,NaN,NaN,A5X5Y0
355,355,HTR3E,5-HT(3E) iso3,A5X5Y0-3,3q27.1,Y20-p,22953108,human,52.81,NaN,...,0,20,A5X5Y0-3_20,NaN,NaN,NaN,NaN,NaN,NaN,A5X5Y0
785,785,AANAT,AA-NAT iso100,Q16613_VAR_A129T,17q25.1,T129-p,3251801,human,23.37,Acetyltransf_1,...,0,129,Q16613_VAR_A129T_129,NaN,NaN,NaN,NaN,NaN,NaN,Q16613_VAR_A129T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435965,436098,MAPT,Tau iso8,P10636-8,17q21.31,S208-gl,18416100,human,45.85,NaN,...,0,208,P10636-8_208,NaN,NaN,NaN,NaN,NaN,NaN,P10636
435966,436099,MAPT,Tau iso8,P10636-8,17q21.31,S238-gl,18416101,human,45.85,NaN,...,0,238,P10636-8_238,NaN,NaN,NaN,NaN,NaN,NaN,P10636
435967,436100,MAPT,Tau iso8,P10636-8,17q21.31,S356-gl,5227900,human,45.85,Tubulin-binding,...,0,356,P10636-8_356,NaN,NaN,NaN,NaN,NaN,NaN,P10636
435968,436101,MAPT,Tau iso8,P10636-8,17q21.31,S400-gl,18416102,human,45.85,NaN,...,0,400,P10636-8_400,NaN,NaN,NaN,NaN,NaN,NaN,P10636


In [24]:
uniprot = "P78559"
interfaces_data = pd.read_csv("/rcfs/projects/proteometer/ProtVar/predictions/interfaces/2024.05.28_interface_summary_5A.tsv", delimiter='\t', header=0)


interface_only_uniprot = interfaces_data.loc[(interfaces_data['uniprot_id1'] == uniprot) | (interfaces_data['uniprot_id2'] == uniprot)] # isolate to uniprot in either 1 or 2


path = glob.glob("/rcfs/projects/proteometer/alphafold_swissprot_pdb/*" + uniprot + "-F1-*") # change this to have F1 using pdb file name

if path:
    "i found it!"

KeyboardInterrupt: 

In [29]:

interfaces_data = pd.read_csv("/rcfs/projects/proteometer/ProtVar/predictions/interfaces/2024.05.28_interface_summary_5A.tsv", delimiter='\t', header=0)
uniprot = null_rows["uniprot_id"].to_list()[0]
interface_only_uniprot = interfaces_data.loc[(interfaces_data['uniprot_id1'] == uniprot) | (interfaces_data['uniprot_id2'] == uniprot)] # isolate to uniprot in either 1 or 2


# parse your structure here
pdb_name = glob.glob("/rcfs/projects/proteometer/alphafold_swissprot_pdb/*" + uniprot + "-F1-*") # change this to have F1 using pdb file name
print(pdb_name)
#print("name of pdb is:", pdb_name)
if len(pdb_name) != 0:  
    ppdb = PandasPdb()  
    ppdb.read_pdb(pdb_name[0])


# for each psp
    for phosphosite_row_index in null_rows.index:
        if pd.notna(null_rows.loc[phosphosite_row_index,'res_number']):
            residue_num = int(null_rows.loc[phosphosite_row_index,'res_number']) # finding the residue number of the psp
            min_dist = np.inf # make min dist extremely high at first
            #print(residue_num)
            # use the residue # to get the coordinates in space from pdb file
            
            
            for interface_index in interface_only_uniprot.index : # get all the residues in all of the interfaces 
                if pd.notna(interface_only_uniprot.loc[interface_index,'ifresid1']) & pd.notna(interface_only_uniprot.loc[interface_index,'ifresid2']):
                    if interfaces_data.loc[interface_index,'uniprot_id1'] == uniprot:
                        interface_residues = interface_only_uniprot.loc[interface_index,'ifresid1']
                    elif interfaces_data.loc[interface_index,'uniprot_id2'] == uniprot:
                        interface_residues = interface_only_uniprot.loc[interface_index,'ifresid2']
                    
                    # check if it's inside of a interface
                    #print(interfaces_data.loc[interface_index,'interaction_id']) 
                    interface_residues = [int(e[1:]) for e in interface_residues.split(",")] # remove the first letter from each bc it includes residue type ormat the interface_residues because it's a string
                    print(interface_residues)
                    if residue_num in interface_residues:
                        #print("found inside interface!")
                        null_rows.loc[phosphosite_row_index,'inside_interface'] = 1 # if residue is in the interface, put 1 in the inside interface column
                        interface_to_add = (interface_only_uniprot.loc[interface_index,'interaction_id'].split('_'))
                        interface_to_add.remove(uniprot)
                        null_rows.loc[phosphosite_row_index,'closest_interface'] = interface_to_add[0] # put unique interfaceID in closest interface
                        null_rows.loc[phosphosite_row_index,'distance_from_interface'] = 0.0 
                        break # break because you don't want to contiue looking for interfaces (and therefore overwrite the inside interface and closest interface)
                    else: # if the phosphosite isn't in any interfaces
                        #print("phosphosite isn't in any interfaces")
                        input_struct = ppdb.df['ATOM']
                        #print(input_struct)
                        
                        #print(residue_num, interface_residues)
                        new_dist = find_mean_distances(input_struct, residue_num, interface_residues)
                        print("the new dist is:" , new_dist)
                        if new_dist:
                            if min_dist > new_dist: # if this is the smallest distance so far, replace min_dist with new_dist
                                interface_to_add = (interface_only_uniprot.loc[interface_index,'interaction_id'].split('_'))
                                interface_to_add.remove(uniprot)
                                null_rows.loc[phosphosite_row_index,'closest_interface'] = interface_to_add[0]
                                null_rows.loc[phosphosite_row_index,'distance_from_interface'] = new_dist # replace distance_from_interface with min_dist
                                min_dist = new_dist
                                print("replaced old dist with", min_dist)

['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-Q9NRA8-F1-model_v4.pdb']


### Debugging inhomogenous error

In [33]:
uniprot = "Q96SD1"
pockets_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/pockets_data.csv")
psp_data = pd.read_csv("/people/imal967/git_repos/pheno_analysis/phosphosite_for_pockets.csv")
Q96QS3_rows = psp_data[psp_data['uniprot_id'] == uniprot]
Q96QS3_rows

psp_only_uniprot = pockets_data.loc[(pockets_data['uniprot'] == uniprot)]
#interface_only_uniprot
#pocket_only_uniprot
pockets_data


,Unnamed: 0,struct_id,pocket_id,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_resid,pocket_plddt_mean,pocket_score_combined_scaled,uniprot_id,full_id
0,0,A0A024R1R8-F1,1,4.042788,0.316535,0.772959,"{21,22,23,24,25,26,28,29,32}",83.937778,283.034096,A0A024R1R8,A0A024R1R8-F1_pocket1
1,1,A0A024R1R8-F1,2,3.175737,0.347111,0.808219,"{12,13,14,15,16,17}",61.206667,102.718057,A0A024R1R8,A0A024R1R8-F1_pocket2
2,2,A0A024RBG1-F1,1,7.310256,0.435597,0.856184,"{2,3,4,5,6,7,8,9,10,18,20,21,22,39,40,41,42,47...",89.456190,979.457587,A0A024RBG1,A0A024RBG1-F1_pocket1
3,3,A0A024RBG1-F1,2,6.350910,0.389675,0.814896,"{54,57,58,60,61,62,64,65,67,68,69,73,74,75,76,...",83.186923,938.222063,A0A024RBG1,A0A024RBG1-F1_pocket2
4,4,A0A024RBG1-F1,3,3.827945,0.378204,0.806045,"{1,2,3,4,5,6,109,110,112,113,114}",77.053636,422.703190,A0A024RBG1,A0A024RBG1-F1_pocket3
...,...,...,...,...,...,...,...,...,...,...,...
547396,547396,X6R8D5-F1,3,3.894257,0.338401,0.777778,"{86,87,88,89,90,91,92,93,98,100,101,102,103}",56.513846,99.047598,X6R8D5,X6R8D5-F1_pocket3
547397,547397,X6R8D5-F1,4,4.196873,0.328247,0.768473,"{43,44,45,46,47,48,49,50,127}",59.902222,107.547205,X6R8D5,X6R8D5-F1_pocket4
547398,547398,X6R8D5-F1,5,4.465454,0.314353,0.751790,"{81,87,90,91,92,93,99,100,101,102,103,104,105}",58.298462,95.185313,X6R8D5,X6R8D5-F1_pocket5
547399,547399,X6R8D5-F1,6,3.198691,0.398147,0.825342,"{66,67,68,69,71,72,73,75,76,77}",61.416000,122.028350,X6R8D5,X6R8D5-F1_pocket6


In [30]:



for phosphosite_row_index in psp_only_uniprot.index:
            #print(psp_only_uniprot)
            #print(phosphosite_row_index)
            residue_num = int(psp_only_uniprot.loc[phosphosite_row_index,'res_number']) # finding the residue number of the psp
            min_dist = np.inf # make min dist extremely high at first
            #print(residue_num)
            # use the residue # to get the coordinates in space from pdb file
            
            
            for pocket_index in pocket_only_uniprot.index : # get all the residues in all of the pockets 
                if pd.notna(pocket_only_uniprot.loc[pocket_index,'pocket_resid']):
                    pocket_residues = pocket_only_uniprot.loc[pocket_index,'pocket_resid']

                    # format the residues
                    pocket_residues = [int(e) for e in pocket_residues[1:-1].split(",")]
                    #print(pocket_residues)
                    if residue_num in pocket_residues:
                        psp_data.loc[phosphosite_row_index,'inside_pocket'] = 1 # if residue is in the pocket, put 1 in the inside pocket column
                        psp_data.loc[phosphosite_row_index,'closest_pocket'] = pocket_only_uniprot.loc[pocket_index,'full_id'] # put unique pocketID in closest pocket
                        psp_data.loc[phosphosite_row_index,'distance_from_pocket'] = 0 
                        break # break because you don't want to contiue looking for pockets (and therefore overwrite the inside pocket and closest pocket)
                    else: # if the phosphosite isn't in any pockets
                        #print("phosphosite isn't in any pockets")
                        input_struct = ppdb.df['ATOM']
                        #print(input_struct)
                        print(residue_num)
                        print(pocket_residues)
                        new_dist = find_mean_distances(input_struct, residue_num, pocket_residues)
                        # print(new_dist)
                        if new_dist:
                            if min_dist > new_dist: # if this is the smallest distance so far, replace min_dist with new_dist
                                psp_data.loc[phosphosite_row_index,'closest_pocket'] = pocket_only_uniprot.loc[pocket_index,'full_id'] # put unique pocketID in closest pocket
                                psp_data.loc[phosphosite_row_index,'distance_from_pocket'] = new_dist # replace distance_from_pocket with min_dist
                                min_dist = new_dist 
                                print("added smallest distance:", min_dist)

['/rcfs/projects/proteometer/alphafold_swissprot_pdb/AF-Q96SD1-F1-model_v4.pdb']
264
[142, 148, 175, 302]
the new dist is: 38.353139955784464
replaced old dist with 38.353139955784464
[267]
the new dist is: 5.226499115086504
replaced old dist with 5.226499115086504
[22, 24, 25, 45, 46, 49, 50, 52, 53, 55, 97, 98, 99, 100, 101, 102]
the new dist is: 49.387338293959544
[487, 488, 489, 492, 493, 494, 495]
the new dist is: 36.230158342867554
317
[142, 148, 175, 302]
the new dist is: 17.290359113760463
replaced old dist with 17.290359113760463
[267]
the new dist is: 28.611872168734436
[22, 24, 25, 45, 46, 49, 50, 52, 53, 55, 97, 98, 99, 100, 101, 102]
the new dist is: 39.04466141881355
[487, 488, 489, 492, 493, 494, 495]
the new dist is: 20.226066323789727
322
[142, 148, 175, 302]
the new dist is: 16.186827661814235
replaced old dist with 16.186827661814235
[267]
the new dist is: 38.81325296854156
[22, 24, 25, 45, 46, 49, 50, 52, 53, 55, 97, 98, 99, 100, 101, 102]
the new dist is: 40.300596

In [28]:
Q96QS3_rows

,Unnamed: 0,GENE,PROTEIN,ACC_ID,HU_CHR_LOC,MOD_RSD,SITE_GRP_ID,ORGANISM,MW_kD,DOMAIN,...,PKA_ID,uniprotID,AA,res_number,pK,state,position,uniprot_id,closest_interface,distance_from_interface
20888,20893,ERAP1,ARTS1,Q9NZ08,5q15,S179-p,18908795,human,107.23,Peptidase_M1_N,...,Q9NZ08_179,Q9NZ08,SER,179.0,NaN,undefined,179.0,Q9NZ08,P01889,49.959375
20889,20894,ERAP1,ARTS1,Q9NZ08,5q15,T185-p,18006846,human,107.23,Peptidase_M1_N,...,Q9NZ08_185,Q9NZ08,THR,185.0,NaN,undefined,185.0,Q9NZ08,P01889,39.612943
20890,20895,ERAP1,ARTS1,Q9NZ08,5q15,S268-p,23084070,human,107.23,NaN,...,Q9NZ08_268,Q9NZ08,SER,268.0,NaN,undefined,268.0,Q9NZ08,P01889,49.370992
20891,20896,ERAP1,ARTS1,Q9NZ08,5q15,Y270-p,10365885,human,107.23,NaN,...,Q9NZ08_270,Q9NZ08,TYR,270.0,10.970566,protonated,270.0,Q9NZ08,P01889,51.053779
20892,20897,ERAP1,ARTS1,Q9NZ08,5q15,Y399-p,23084076,human,107.23,Peptidase_M1,...,Q9NZ08_399,Q9NZ08,TYR,399.0,10.877961,protonated,399.0,Q9NZ08,P01889,28.903107
20893,20898,ERAP1,ARTS1,Q9NZ08,5q15,Y451-p,15568589,human,107.23,Peptidase_M1,...,Q9NZ08_451,Q9NZ08,TYR,451.0,11.309102,protonated,451.0,Q9NZ08,P01889,13.792920
20894,20899,ERAP1,ARTS1,Q9NZ08,5q15,Y464-p,23084079,human,107.23,Peptidase_M1,...,Q9NZ08_464,Q9NZ08,TYR,464.0,NaN,undefined,464.0,Q9NZ08,P01889,26.148572
20895,20900,ERAP1,ARTS1,Q9NZ08,5q15,T473-p,23084073,human,107.23,Peptidase_M1,...,Q9NZ08_473,Q9NZ08,THR,473.0,NaN,undefined,473.0,Q9NZ08,P01889,27.308463
20896,20901,ERAP1,ARTS1,Q9NZ08,5q15,T559-p,55379735,human,107.23,NaN,...,Q9NZ08_559,Q9NZ08,THR,559.0,NaN,undefined,559.0,Q9NZ08,P01889,16.501246
20897,20902,ERAP1,ARTS1,Q9NZ08,5q15,Y608-p,482896,human,107.23,ERAP1_C,...,Q9NZ08_608,Q9NZ08,TYR,608.0,11.547349,protonated,608.0,Q9NZ08,P01889,13.782251
